# Single-Qubit Systems
<hr></hr>

In the introductory chapter to [quantum bits and circuits](https://learnquantum.io/chapters/02_quantum_computing/02_01_bits_to_qubits.html), we presented the model of a qubit inspired by the probabilistic behavior of the spin of an electron. However, we only described how the electron behaves when its spin was pointing in the $\texttt{+}z, \texttt{-}z, \texttt{+}x, \texttt{-}x$ directions (described by states $|0\rangle, |1\rangle, |+\rangle, |-\rangle$, respectively). 

But what if the spin is pointing along some arbitrary angle with respect to the $\texttt{+}z$ direction? We did mention that the probability of measuring spin-up or spin-down depends on this angle, but did not give a quantitative rule for how electrons behave. 

In this chapter, we will not only expand on this idea, but will generalize the state of the spin of an electron pointing in any direction, which represents the most general way to express the state of a qubit. We will then discuss the different operations we can apply to a qubit to transform it from one arbitrary state to another, and lastly talk about the Bloch Sphere, which is a graphical representation of the mathematical description of the qubit's state.

## 1. Generalizing the Qubit


### 1.1 Real Probability Amplitudes

If we, once again, look at the Stern-Gerlach experiment, observations of repeated experiments show that for an electron with its spin at an angle $\theta$ with respect to the $\texttt{+}z$ axis, the probability of the electron deflecting in the $\texttt{+}z$ direction (i.e., measuring state $|0\rangle$) is given by the cosine squared of the angle $\theta/2$:

$$ P_{0} = \cos^2\left(\frac{\theta}{2}\right) $$

Similarly, the probability of the electron deflecting in the $\texttt{-}z$ direction (i.e., measuring state $|1\rangle$) is equal the sine squared of the angle:

$$ P_{1} = \sin^2\left(\frac{\theta}{2}\right) $$

<img src="images/02_03_01_stern-gerlach_angle_prob.png" align = "center" width="360"/>

From a probability standpoint, these expressions work since the sum of sine and cosine squared for any common angle always add to one, just like probabilities ought to do:

$$
\begin{aligned}
P_{0} &+ P_{1} = 1
\\
\\
\cos^2\left(\frac{\theta}{2}\right) &+ \sin^2\left(\frac{\theta}{2}\right) = 1
\end{aligned}
$$

Furthermore, we can take this one step farther, and use the same approach we followed when going from [probabilities to probability amplitudes](https://learnquantum.io/chapters/02_quantum_computing/02_01_bits_to_qubits.html#from-probabilities-to-probability-amplitudes) and show that the state of a qubit (electron spin) for an arbitrary angle $\theta$ is given by the statevector:

$$ |q\rangle = \cos\left(\frac{\theta}{2}\right)|0\rangle + \sin\left(\frac{\theta}{2}\right) |1\rangle $$

And this, of course, checks out for the four cases we have explored so far:

| Spin Direction | State | Angle $\theta$ [rad] | $\cos\left(\frac{\theta}{2}\right)$ |  $\sin\left(\frac{\theta}{2}\right)$ |
| :-: | :-: | :-: | :-: | :-: |
| $\texttt{+}z$ | $|0\rangle$ | $0$ | $1$ | $0$ |
| $\texttt{-}z$ | $|1\rangle$ | $\pi$ |  $0$ | $1$ |
| $\texttt{+}x$ | $|+\rangle$ | $\frac{\pi}{2}$ | $\frac{1}{\sqrt{2}}$ | $\frac{1}{\sqrt{2}}$ |
| $\texttt{-}x$ | $|-\rangle$ | $\frac{3\pi}{2}$ | $\frac{1}{\sqrt{2}}$ | $-\frac{1}{\sqrt{2}}$ |

But more generally, it works for any angle. Let's look at the specific example where $\theta = \pi/3$. The expectation is that we will get the following respective probabilities for measuring states $|0\rangle$ and $|1\rangle$:

$$ 
\begin{aligned}
&P_{0} = \cos^2\left(\frac{\pi/3}{2}\right) = \left(\frac{\sqrt{3}}{2}\right)^2 = \frac{3}{4}
\\
\\
&P_{1} = \sin^2\left(\frac{\pi/3}{2}\right) = \left(\frac{1}{2}\right)^2 = \frac{1}{4}
\end{aligned}
$$


Let is now use the `Statevector` class in qiskit to construct this statevector, and see if the measurement probabilities match our expectation:

In [1]:
import numpy as np
from qiskit.quantum_info import Statevector

θ = np.pi/3               # Spin angle wrt to +z axis
α = np.cos(θ/2)           # Probability amplitude associated with |0⟩
β = np.sin(θ/2)           # Probability amplitude associated with |1⟩

q = Statevector([α, β])   # Construct statevector |q⟩ = α|0⟩ + β|1⟩ = [α β]ᵀ

probs = q.probabilities() # Extract expected probs array [p₀, p₁]
print(probs)

[0.75 0.25]


The result above is the calculation of the ideal probabilities, but we can also sample the statevector (as if we were performing measurements in this system) and get a distribution of results that should be approximately give us $75\%$ of measuring $0$ and $25\%$ of measuring $1$:

In [5]:
counts = q.sample_counts(shots=1000) # "Measure" statevector 1000 times and 
                                     # return dict {'0': num_of_0s, '1': num_of_0s}
print(counts)

{'0': 730, '1': 270}


This generalization of the qubit, where the probability amplitudes are given by the $\cos$ and $\sin$ squared of the angle $\theta/2$, works well to describe the results of the Stern-Gerlach experiment; however, there is a bit of an issue. In this particular case, we are only concerned about the direction of the spin on the $xz$-plane because we are shooting the electrons along the $y$ direction. But how do we describe the spin of the electron if it is aligned with, let's say, the $\texttt{+}y$ axis? Well, experiments show that, if we were to measure the spin of such an electron along the $z$ axis, we will still see the electron deflecting up or down with $50\%$ probability. As a matter of fact, this is true no matter which orientation (given by the angle $\varphi$) the electron has in the $xy$ plane:

<img src="images/02_03_02_elec_phi_angle.png" align = "center" width="520"/>

So, how do we re